In [1]:
import numpy as np, pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
cd drive

/content/drive


In [7]:
cd My Drive

/content/drive/My Drive


In [138]:
%ls pyghack_data/

champaign_data.py         land_area.csv            us-counties_covid19.csv
Champaign_zipcode.gsheet  mask-use-by-county.csv   us-counties_coviddeaths.csv
Education.csv             PopulationEstimates.csv  us_pop_data.csv
il.1990_2018.19ages.txt   PovertyEstimates.csv
illinois_population.csv   Unemployment.csv


In [27]:
df_unem = pd.read_csv('pyghack_data/Unemployment.csv')
df_unem = df_unem[['FIPStxt','Unemployment_rate_2019','Median_Household_Income_2018']]

In [33]:
df_poverty = pd.read_csv('/content/drive/My Drive/pyghack_data/PovertyEstimates.csv')
df_poverty = df_poverty[['FIPStxt','PCTPOVALL_2018']]

In [53]:
df_edu = pd.read_csv('/content/drive/My Drive/pyghack_data/Education.csv')

In [54]:
df_edu.rename(columns = {'FIPS Code':'FIPStxt'}, inplace = True)

In [65]:
df_land = pd.read_csv('/content/drive/My Drive/pyghack_data/land_area.csv')

In [71]:
df_tpop = pd.read_csv('/content/drive/My Drive/pyghack_data/PopulationEstimates.csv')

In [113]:
df_features1 = df_unem
df_features1 = pd.merge(df_features1,df_poverty,how='inner', on ='FIPStxt')
df_features1 = pd.merge(df_features1,df_edu,how='inner', on ='FIPStxt')
df_features1 = pd.merge(df_features1,df_land,how='inner', on ='FIPStxt')
df_features1 = pd.merge(df_features1,df_tpop,how='inner', on ='FIPStxt')
df_features1.pop('Unnamed: 2')
df_features1.rename(columns = {'FIPStxt':'fips','Unemployment_rate_2019':'Unemployment_rate','Median_Household_Income_2018':'Median_Household_Income','PCTPOVALL_2018':'poverty_rate','POP_ESTIMATE_2019':'total_pop'}, inplace = True)
df_features1['Median_Household_Income']=df_features1['Median_Household_Income'].apply(lambda x: ''.join(c for c in x if c.isnumeric()))
df_features1['total_pop']=df_features1['total_pop'].apply(lambda x: ''.join(c for c in x if c.isnumeric()))
df_features1 = df_features1.astype({'fips':int, 'Unemployment_rate':float, 'Median_Household_Income':float,
       'poverty_rate':float, 'no_edu':float, 'high_school':float, 'college_drop':float, 'college':float, 'land_area':float, 'total_pop':float})
df_features1['pop_density'] = df_features1['total_pop']/df_features1['land_area']

In [114]:
df_features1

,fips,Unemployment_rate,Median_Household_Income,poverty_rate,no_edu,high_school,college_drop,college,land_area,total_pop,pop_density
0,0,3.7,61937.0,13.1,12.3,27.1,29.0,31.5,3531905.43,328239523.0,92.935536
1,1000,3.0,49881.0,16.8,14.2,30.9,29.9,24.9,50645.33,4903185.0,96.814158
2,1001,2.7,59338.0,13.8,11.3,32.6,28.4,27.7,594.44,55869.0,93.985936
3,1003,2.7,57588.0,9.8,9.7,27.6,31.3,31.3,1589.78,223234.0,140.418171
4,1005,3.8,34382.0,30.9,27.0,35.7,25.1,12.2,884.88,24686.0,27.897568
...,...,...,...,...,...,...,...,...,...,...,...
3186,56037,3.9,73315.0,8.4,9.0,33.3,35.7,22.0,10426.65,42343.0,4.061036
3187,56039,2.7,99087.0,6.3,5.6,13.2,23.8,57.4,3995.38,23464.0,5.872783
3188,56041,3.9,63401.0,10.0,7.2,40.1,37.2,15.4,2081.26,20226.0,9.718152
3189,56043,3.9,55190.0,11.9,10.4,30.3,37.7,21.5,2238.55,7805.0,3.486632


In [133]:
maskdf = pd.read_csv('/content/drive/My Drive/pyghack_data/mask-use-by-county.csv')
maskdf = maskdf.rename(columns={"COUNTYFP": "fips"})
maskdf['fips'] = maskdf['fips'].astype(int)
cdeath_df = pd.read_csv('/content/drive/My Drive/pyghack_data/us-counties_coviddeaths.csv')
coviddf = cdeath_df.merge(maskdf, on=['fips'],how='inner')
coviddf['fips'] = coviddf['fips'].astype(int)
coviddf = coviddf.drop(columns=['date'])
coviddf.pop('county')
coviddf.pop('state')
coviddf['no_mask']=coviddf['NEVER']+coviddf['RARELY']

In [134]:
df_features2 = pd.merge(df_features1,coviddf,how='inner', on ='fips')

In [136]:
df_features2['death_rate'] = df_features2['deaths']/df_features2['total_pop']

In [137]:
df_features2

,fips,Unemployment_rate,Median_Household_Income,poverty_rate,no_edu,high_school,college_drop,college,land_area,total_pop,pop_density,cases,deaths,confirmed_cases,confirmed_deaths,probable_cases,probable_deaths,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS,no_mask,death_rate
0,1001,2.7,59338.0,13.8,11.3,32.6,28.4,27.7,594.44,55869.0,93.985936,1738,25.0,1565.0,24.0,173.0,1.0,0.053,0.074,0.134,0.295,0.444,0.127,0.000447
1,1003,2.7,57588.0,9.8,9.7,27.6,31.3,31.3,1589.78,223234.0,140.418171,5165,50.0,4979.0,46.0,186.0,4.0,0.083,0.059,0.098,0.323,0.436,0.142,0.000224
2,1005,3.8,34382.0,30.9,27.0,35.7,25.1,12.2,884.88,24686.0,27.897568,857,7.0,657.0,7.0,200.0,0.0,0.067,0.121,0.120,0.201,0.491,0.188,0.000284
3,1007,3.1,46064.0,21.8,16.8,47.3,24.4,11.5,622.58,22394.0,35.969675,642,10.0,611.0,6.0,31.0,4.0,0.020,0.034,0.096,0.278,0.572,0.054,0.000447
4,1009,2.7,50412.0,13.2,19.8,34.0,33.5,12.6,644.78,57826.0,89.683303,1594,15.0,1236.0,15.0,358.0,0.0,0.053,0.114,0.180,0.194,0.459,0.167,0.000259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3119,56037,3.9,73315.0,8.4,9.0,33.3,35.7,22.0,10426.65,42343.0,4.061036,328,2.0,310.0,NaN,18.0,NaN,0.061,0.295,0.230,0.146,0.268,0.356,0.000047
3120,56039,2.7,99087.0,6.3,5.6,13.2,23.8,57.4,3995.38,23464.0,5.872783,529,1.0,496.0,NaN,33.0,NaN,0.095,0.157,0.160,0.247,0.340,0.252,0.000043
3121,56041,3.9,63401.0,10.0,7.2,40.1,37.2,15.4,2081.26,20226.0,9.718152,347,2.0,283.0,NaN,64.0,NaN,0.098,0.278,0.154,0.207,0.264,0.376,0.000099
3122,56043,3.9,55190.0,11.9,10.4,30.3,37.7,21.5,2238.55,7805.0,3.486632,112,6.0,103.0,NaN,9.0,NaN,0.204,0.155,0.069,0.285,0.287,0.359,0.000769


In [151]:
df_hr_uspop = pd.read_csv('/content/drive/My Drive/pyghack_data/us_pop_data.csv')
df_hr_uspop.pop('non-hispanic')
df_hr_uspop.pop('population')
df_hr_uspop.rename(columns = {'county':'fips','asian / pacific islander':'asian','amerian indian/alaska native':'native','PCTPOVALL_2018':'poverty_rate','POP_ESTIMATE_2019':'total_pop'}, inplace = True)
df_hr_uspop.keys()
df_feat = pd.merge(df_features2,df_hr_uspop,how='inner', on ='fips')

In [153]:
df_feat.keys()

Index(['fips', 'Unemployment_rate', 'Median_Household_Income', 'poverty_rate',
       'no_edu', 'high_school', 'college_drop', 'college', 'land_area',
       'total_pop', 'pop_density', 'cases', 'deaths', 'confirmed_cases',
       'confirmed_deaths', 'probable_cases', 'probable_deaths', 'NEVER',
       'RARELY', 'SOMETIMES', 'FREQUENTLY', 'ALWAYS', 'no_mask', 'death_rate',
       'black', 'white', 'asian', 'native', 'hispanic', '0', '0-4', '12-16',
       '16-20', '20-24', '24-28', '28-32', '32-36', '36-40', '4-8', '40-44',
       '44-48', '48-52', '52-56', '56-60', '60-64', '64-68', '68-72', '8-12',
       'Female', 'Male'],
      dtype='object')

In [156]:
df_train=df_feat[['Unemployment_rate', 'Median_Household_Income', 'poverty_rate',
       'no_edu', 'high_school', 'college_drop', 'college', 'land_area',
       'total_pop', 'pop_density','black', 'white', 'asian', 'native', 'hispanic', '0', '0-4', '12-16',
       '16-20', '20-24', '24-28', '28-32', '32-36', '36-40', '4-8', '40-44',
       '44-48', '48-52', '52-56', '56-60', '60-64', '64-68', '68-72', '8-12',
       'Female', 'Male']]

In [157]:
df_train

,Unemployment_rate,Median_Household_Income,poverty_rate,no_edu,high_school,college_drop,college,land_area,total_pop,pop_density,black,white,asian,native,hispanic,0,0-4,12-16,16-20,20-24,24-28,28-32,32-36,36-40,4-8,40-44,44-48,48-52,52-56,56-60,60-64,64-68,68-72,8-12,Female,Male
0,2.7,59338.0,13.8,11.3,32.6,28.4,27.7,594.44,55869.0,93.985936,0.202047,0.777900,0.015162,0.004892,0.029658,0.011636,0.048866,0.067535,0.057067,0.066635,0.062247,0.065251,0.065035,0.071330,0.061564,0.068542,0.070628,0.058218,0.046564,0.041564,0.031276,0.020557,0.015665,0.069819,0.514487,0.485513
1,2.7,57588.0,9.8,9.7,27.6,31.3,31.3,1589.78,223234.0,140.418171,0.093192,0.884649,0.013462,0.008696,0.046468,0.010343,0.044491,0.059315,0.050949,0.057201,0.055302,0.060343,0.059063,0.066328,0.058719,0.065324,0.072213,0.071851,0.065438,0.057201,0.038826,0.023227,0.019741,0.064127,0.515384,0.484616
2,3.8,34382.0,30.9,27.0,35.7,25.1,12.2,884.88,24686.0,27.897568,0.491861,0.494273,0.006632,0.007234,0.042764,0.010289,0.041317,0.058318,0.059162,0.072184,0.067119,0.061533,0.058277,0.065271,0.058197,0.064427,0.067320,0.061734,0.061895,0.053736,0.036454,0.023150,0.018970,0.060649,0.472168,0.527832
3,3.1,46064.0,21.8,16.8,47.3,24.4,11.5,622.58,22394.0,35.969675,0.217054,0.774509,0.004018,0.004420,0.026250,0.010982,0.046384,0.056696,0.058170,0.077054,0.071473,0.067500,0.064286,0.071339,0.052277,0.070580,0.068482,0.060848,0.052768,0.044286,0.032634,0.018839,0.016205,0.059196,0.467813,0.532188
4,2.7,50412.0,13.2,19.8,34.0,33.5,12.6,644.78,57826.0,89.683303,0.019225,0.968568,0.005187,0.007019,0.095712,0.011705,0.048115,0.063416,0.054945,0.061618,0.058264,0.060754,0.061428,0.067790,0.062846,0.067376,0.068240,0.063676,0.056743,0.051193,0.034059,0.022528,0.017842,0.067462,0.507261,0.492739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3118,3.9,73315.0,8.4,9.0,33.3,35.7,22.0,10426.65,42343.0,4.061036,0.018211,0.950547,0.012845,0.018397,0.160833,0.012683,0.055818,0.070033,0.058698,0.065782,0.074121,0.078651,0.065922,0.057978,0.073285,0.055330,0.066874,0.066804,0.048013,0.031707,0.019349,0.011103,0.010941,0.076909,0.485447,0.514553
3119,2.7,99087.0,6.3,5.6,13.2,23.8,57.4,3995.38,23464.0,5.872783,0.010182,0.963000,0.016724,0.010095,0.148780,0.009965,0.039080,0.042589,0.046705,0.081842,0.087864,0.089468,0.075213,0.073090,0.054244,0.061652,0.065205,0.066852,0.057710,0.042589,0.025389,0.014384,0.014081,0.052077,0.484034,0.515966
3120,3.9,63401.0,10.0,7.2,40.1,37.2,15.4,2081.26,20226.0,9.718152,0.011675,0.965466,0.006552,0.016306,0.092369,0.013498,0.059707,0.071826,0.049214,0.056850,0.061185,0.071925,0.061973,0.055766,0.080546,0.052613,0.067294,0.070742,0.055077,0.037736,0.023006,0.012414,0.012562,0.086063,0.493325,0.506675
3121,3.9,55190.0,11.9,10.4,30.3,37.7,21.5,2238.55,7805.0,3.486632,0.011034,0.960304,0.009639,0.019023,0.140520,0.009131,0.042359,0.070133,0.044642,0.046037,0.052758,0.061763,0.055802,0.058339,0.057324,0.065694,0.071909,0.076221,0.065568,0.053266,0.040710,0.026633,0.030691,0.071021,0.494230,0.505770
